#Library and functions

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from collections import Counter
import PIL
import glob
from tqdm import tqdm
import networkx as nx

#Graphics
from scipy.stats import gaussian_kde
from scipy.stats import norm
import seaborn as sns
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

#NN
from tensorflow import keras
from tensorflow.keras import layers, callbacks
#import tensorflow as tf

#sklearn modules: evaluation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, classification_report, RocCurveDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Recurrence plots
from pyts.image import RecurrencePlot
from pyts.datasets import load_gunpoint
from mpl_toolkits.axes_grid1 import ImageGrid

# Neural net training: regression

In [ ]:
#Read all pngs from the train set
pngs = glob.glob('your_path/delta_*/*png')

ims = {}
for png in pngs:
  ims[png]=np.array(PIL.Image.open(png).convert('L'))

In [ ]:
#Questions = images
#Solutions = a parameter that generated that image

questions = np.array([each for each in ims.values()]).astype(np.float32)
solutions = np.array([float(each.split('_')[-1].split('.')[0].split('/')[1])/(1e16) for each in ims]).astype(np.float32)

##Architecture

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.MaxPooling2D(2),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Conv2D(3, 3, activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(units=1024, activation='relu'),
  keras.layers.Dense(units=512, activation='relu'),
  keras.layers.Dense(units=256, activation='relu'),
  keras.layers.Dense(units=64, activation='relu'),
  keras.layers.Dense(units=1)
])

model.compile(loss='mean_squared_error', optimizer="adam")

## Training

In [ ]:
N_size = 100

#Train/test split (gnuplot data):
X_train, X_test, y_train, y_test = train_test_split(questions, solutions, test_size=0.3, random_state=42)

X_train = X_train.reshape(-1, N_size, N_size, 1)
X_test = X_test.reshape(-1, N_size, N_size, 1)

In [ ]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

##Testing

In [ ]:
test_pngs = glob.glob('your_path/test/*png')
test_ims = {}

for png in test_pngs:
  test_ims[png]=np.array(PIL.Image.open(png).convert('L'))


#Prepare test questions and solutions as before:
test_questions = np.array([each for each in test_ims.values()]).astype(np.float32)
test_solutions = np.array([float(each.split('_')[1].split('.')[0].split('/')[2])/(1e16)
                            for each in test_ims]).astype(np.float32)

#Apply model:
test_answers = model.predict(test_questions)

In [ ]:
df_res = pd.DataFrame(data={'test_actual': test_solutions, 'test_predictions': test_answers.flatten(), 'N': np.repeat(100, len(test_answers))})
df_res.to_csv('NN_RP_standardmap_scatter_reg_n_100_epsE9_1k_test_40k_train_batch100.csv')